In [25]:
import sqlite3 as sql_client
import pandas as data_analyzer

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [26]:
SQL_DB_FILE_PATH='./FPA_FOD_20170508.sqlite'

causes_map = {
  'Miscellaneous': 0,
  'Lightning': 1,
  'Debris Burning': 2,
  'Campfire': 3,
  'Equipment Use': 4,
  'Arson': 5,
  'Children': 6,
  'Railroad': 7,
  'Smoking': 8,
  'Powerline': 9,
  'Structure': 10,
  'Fireworks': 11,
  'Missing/Undefined': 12
}


In [27]:
def set_causes_map(data_frame, description):
  causes_set = set(data_frame['cause'])
  causes_map = dict(zip(causes_set, range(len(causes_set))))

In [28]:
def display_rows(data_frame, description):
  print('='*88)
  print('Description: ' + description)
  print('Result:')
  print(data_frame.to_string())

In [29]:
def train_and_test(data_frame, description):
  data_frame = data_frame.dropna()
  data_frame['cause'] = data_frame['cause'].map(causes_map)
  X = data_frame.drop(['cause'], axis=1).values
  Y = data_frame['cause'].values
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=0) # 80% to train; 20% test
  classifier = RandomForestClassifier(n_estimators=200).fit(X_train, Y_train)
  score = classifier.score(X_test, Y_test) * 100
  print('Test Set Score: {} %'.format(score))


In [30]:
queries_and_executors = [
  [
    """
    SELECT fire_year, COUNT(*) as incident_count
    FROM fires
    GROUP BY fire_year
    ORDER BY fire_year ASC
    """, 
    'Have wildfires become more or less frequent over time?',
    display_rows
  ],
  [
    """
    SELECT county, COUNT(*) as incident_count
    FROM fires
    GROUP BY county
    ORDER BY incident_count DESC
    """,
    'What counties are the most and least fire-prone?',
    display_rows
  ],
  # distinct query will read the entire table, better to use it once and update the local values
  [
    """
    SELECT stat_cause_descr as cause
    FROM fires
    LIMIT 100
    """, 
    'All Causes',
    set_causes_map
  ],
  [
    """
    SELECT
      latitude,
      longitude,
      fire_size as size,
      strftime('%w', discovery_date) as day_of_week,
      strftime('%m', discovery_date) as month,
      fire_year as year,
      stat_cause_descr as cause
    FROM fires
    LIMIT 50000
    """, 
    'Sample Data',
    train_and_test
  ]
]


In [31]:
connection = sql_client.connect(SQL_DB_FILE_PATH)
for query_description_executor in queries_and_executors:
  query = query_description_executor[0]
  description = query_description_executor[1]
  executor = query_description_executor[2]
  data_frame = data_analyzer.read_sql(query, con=connection)
  executor(data_frame, description)

Description: Have wildfires become more or less frequent over time?
Result:
    FIRE_YEAR  incident_count
0        1992           67975
1        1993           61989
2        1994           75955
3        1995           71472
4        1996           75574
5        1997           61450
6        1998           68370
7        1999           89363
8        2000           96416
9        2001           86587
10       2002           75656
11       2003           68261
12       2004           69279
13       2005           88604
14       2006          114004
15       2007           95573
16       2008           85378
17       2009           78325
18       2010           79889
19       2011           90552
20       2012           72769
21       2013           64780
22       2014           67753
23       2015           74491
Description: What counties are the most and least fire-prone?
Result:
                                                  COUNTY  incident_count
0                              

Test Set Score: 69.52000000000001 %
